In [2]:
import pandas as pd

books_df = pd.read_csv('books.csv')        # book info dataset
customers_df = pd.read_csv('customers.csv')  # customer info dataset

In [3]:
import hashlib
import random
from datetime import datetime, timedelta

def generate_order(customer, book):
    # Remove (+84) prefix
    customer_phone_clean = customer['customer_phone'].replace('(+84)', '').replace(' ', '')
    
    # Order date (random date in the last 30 days)
    order_date = datetime.today() - timedelta(days=random.randint(0, 30))
    date_id = int(order_date.strftime('%Y%m%d'))
    
    # Order quantity weighted (1 is more likely)
    weights = [100, 50, 25, 15, 10, 5, 1]
    order_quantity = random.choices(range(1, 8), weights=weights)[0]
    
    # Generate order_id as hash % 1000000
    order_id = int(hashlib.md5((customer_phone_clean + str(date_id) + book['book_id']).encode()).hexdigest(), 16) % 1000000
    
    # Generate payment_id per customer per day (same for all orders of that customer on the same day)
    last3_phone = customer_phone_clean[-3:].rjust(3, "0")  # pad if needed
    mm_str = str(date_id)[4:6]  # month part
    sum_ord = sum(ord(c) for c in last3_phone + mm_str)
    payment_id = sum_ord % 4 + 1
    return {
        'order_id': order_id,
        'customer_phone': customer_phone_clean,
        'book_id': book['book_id'],
        'order_date': order_date.strftime('%Y-%m-%d'),
        'date_id': date_id,
        'order_quantity': order_quantity,
        'payment_id': payment_id
    }

orders = []

for _, customer in customers_df.sample(20).iterrows():  # pick 20 random customers
    books_sample = books_df.sample(random.randint(1, 3))  # each customer buys 1-3 books
    for _, book in books_sample.iterrows():
        orders.append(generate_order(customer, book))

orders_df = pd.DataFrame(orders)
print(orders_df.head())


   order_id customer_phone       book_id  order_date   date_id  \
0    213398      821864644  a2n2EAAAQBAJ  2025-10-25  20251025   
1    196677      905528664  4ZctAAAAQBAJ  2025-11-05  20251105   
2     12751      906353129  0rhp0QEACAAJ  2025-10-24  20251024   
3    913920      906353129  9S_KDwAAQBAJ  2025-10-21  20251021   
4    222213      906353129  GRbREAAAQBAJ  2025-11-07  20251107   

   order_quantity  payment_id  
0               1           4  
1               1           3  
2               1           2  
3               1           2  
4               2           3  


In [4]:
import requests
API_URL = 'https://api.mockaroo.com/api/288bcd00?count=40&key=97b6ba60'

for _, order in orders_df.iterrows():
    response = requests.post(API_URL)  # send as JSON
    if response.status_code == 200:
        print(f"Order {order['order_id']} sent successfully.")
    else:
        print(f"Failed to send order {order['order_id']}: {response.text}")


Order 213398 sent successfully.
Order 196677 sent successfully.
Order 12751 sent successfully.
Order 913920 sent successfully.
Order 222213 sent successfully.
Order 5383 sent successfully.
Order 561375 sent successfully.
Order 870464 sent successfully.
Order 167302 sent successfully.
Order 747252 sent successfully.
Order 489368 sent successfully.
Order 397586 sent successfully.
Order 254094 sent successfully.
Order 344023 sent successfully.
Order 24506 sent successfully.
Order 472594 sent successfully.
Order 546552 sent successfully.
Order 909469 sent successfully.
Order 403846 sent successfully.
Order 714153 sent successfully.
Order 219458 sent successfully.
Order 474478 sent successfully.
Order 195043 sent successfully.
Order 199994 sent successfully.
Order 864147 sent successfully.
Order 285400 sent successfully.
Order 132640 sent successfully.
Order 647545 sent successfully.
Order 770161 sent successfully.
Order 584442 sent successfully.
Order 764077 sent successfully.
Order 189927

In [ ]:
import requests
import pandas as pd
from io import StringIO

mockaroo_url = "https://api.mockaroo.com/api/288bcd00?count=40&key=97b6ba60"
response = requests.get(mockaroo_url)

if response.status_code == 200:
    data = response.text
    df = pd.read_csv(StringIO(data))
    print(df)
else:
    print(f"Error: {response.status_code}")


    customer_phone       book_id  order_date   date_id  order_quantity  \
0   (+84)902315817  ZOfnEAAAQBAJ  2025/11/14  20251114               1   
1   (+84)926682752  YEEAEAAAQBAJ  2025/11/14  20251114               1   
2   (+84)821864644  B4oZEQAAQBAJ  2025/11/14  20251114               1   
3   (+84)973738512  oGIsEQAAQBAJ  2025/11/14  20251114               1   
4   (+84)942758339  r5-mAwAAQBAJ  2025/11/14  20251114               2   
5   (+84)829158296  Hif4DwAAQBAJ  2025/11/14  20251114               1   
6   (+84)771773772  IynFEAAAQBAJ  2025/11/14  20251114               4   
7   (+84)821864644  eENiEQAAQBAJ  2025/11/14  20251114               1   
8   (+84)943337972  Hif4DwAAQBAJ  2025/11/14  20251114               1   
9   (+84)821864644  ph5pEQAAQBAJ  2025/11/14  20251114               1   
10  (+84)906446757  Eb9IEAAAQBAJ  2025/11/14  20251114               1   
11  (+84)906353129  zCuODwAAQBAJ  2025/11/14  20251114               1   
12  (+84)775651593  DhLGDwAAQBAJ  2025